In [1]:
# importing the tensorflow package
import tensorflow as tf

In [2]:
tf.test.is_built_with_cuda()

True

In [113]:
tf.config.list_physical_devices('GPU')

[]

# Simple RNN (numpy)

In [80]:
# в первую очередь подключим numpy и библиотеку copy, которая понадобится, чтобы сделать deepcopy ряда элементов
import numpy as np
np.random.seed(0)

In [81]:
# вычислим сигмоиду
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# конвертируем значение функции сигмоиды в ее производную. 
def sigmoid_output_to_derivative(output):
    return output*(1-output)

In [82]:
# генерация тренировочного датасета

int2binary = {}
binary_dim = 8

largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([list(range(largest_number))],dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary[i] = binary[i]

In [83]:
int2binary

{0: array([0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 1: array([0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8),
 2: array([0, 0, 0, 0, 0, 0, 1, 0], dtype=uint8),
 3: array([0, 0, 0, 0, 0, 0, 1, 1], dtype=uint8),
 4: array([0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8),
 5: array([0, 0, 0, 0, 0, 1, 0, 1], dtype=uint8),
 6: array([0, 0, 0, 0, 0, 1, 1, 0], dtype=uint8),
 7: array([0, 0, 0, 0, 0, 1, 1, 1], dtype=uint8),
 8: array([0, 0, 0, 0, 1, 0, 0, 0], dtype=uint8),
 9: array([0, 0, 0, 0, 1, 0, 0, 1], dtype=uint8),
 10: array([0, 0, 0, 0, 1, 0, 1, 0], dtype=uint8),
 11: array([0, 0, 0, 0, 1, 0, 1, 1], dtype=uint8),
 12: array([0, 0, 0, 0, 1, 1, 0, 0], dtype=uint8),
 13: array([0, 0, 0, 0, 1, 1, 0, 1], dtype=uint8),
 14: array([0, 0, 0, 0, 1, 1, 1, 0], dtype=uint8),
 15: array([0, 0, 0, 0, 1, 1, 1, 1], dtype=uint8),
 16: array([0, 0, 0, 1, 0, 0, 0, 0], dtype=uint8),
 17: array([0, 0, 0, 1, 0, 0, 0, 1], dtype=uint8),
 18: array([0, 0, 0, 1, 0, 0, 1, 0], dtype=uint8),
 19: array([0, 0, 0, 1, 0, 0, 1, 1], dtyp

In [84]:
# входные переменные
alpha = 0.1
input_dim = 2
hidden_dim = 16
output_dim = 1

In [85]:
# инициализация весов нейронной сети
synapse_0 = 2*np.random.random((input_dim,hidden_dim)) - 1
synapse_1 = 2*np.random.random((hidden_dim,output_dim)) - 1
synapse_h = 2*np.random.random((hidden_dim,hidden_dim)) - 1

synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

In [86]:
# тренировочная логика
for j in range(10000):
    # генерация простой проблемы сложения (a + b = c)
    a_int = np.random.randint(largest_number/2) # int version
    a = int2binary[a_int] # бинарное кодирование

    b_int = np.random.randint(largest_number/2) # int version
    b = int2binary[b_int] # бинарное кодирование

    # правильный ответ
    c_int = a_int + b_int
    c = int2binary[c_int]

    # место где мы располагаем результаты модели(бинарно закодированные)
    d = np.zeros_like(c)

    overallError = 0
    
    layer_2_deltas = list()
    # здесь будет храниться внутреннее состояние
    layer_1_values = list()
    layer_1_values.append(np.zeros(hidden_dim))
    
    # движение вдоль позиций бинарной кодировки
    for position in range(binary_dim):
        
        # генерация input и output
        X = np.array([[a[binary_dim - position - 1], b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T

        # внутренний слой (input ~+ предыдущий внутренний)
        layer_1 = sigmoid(np.dot(X, synapse_0) + np.dot(layer_1_values[-1], synapse_h))

        # output layer (новое бинарное представление)
        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # подсчет ошибки и дельты для вычисления градиентов
        layer_2_error = y - layer_2
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
        overallError += np.abs(layer_2_error[0])
    
        # декодируем оценку чтобы мы могли ее вывести на экран
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        
        # сохраняем внутренний слой, чтобы мы могли его использовать в след. timestep
        layer_1_values.append(layer_1.copy())
    
    # выполним обратный градиентный спуск
    future_layer_1_delta = np.zeros(hidden_dim)
    for position in range(binary_dim):
        X = np.array([[a[position], b[position]]])
        layer_1 = layer_1_values[-position-1]
        prev_layer_1 = layer_1_values[-position-2]
        
        # величина ошибки в output layer
        layer_2_delta = layer_2_deltas[-position-1]
        # величина ошибки в hidden layer
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)

        # обновление всех весов и пробуем заново
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        
        future_layer_1_delta = layer_1_delta
    

    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha    

    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    
    # вывод на экран процесса обучения
    if(j % 1000 == 0):
        print("Error:" + str(overallError))
        print("Pred:" + str(d))
        print("True:" + str(c))
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("------------")


Error:[3.45638663]
Pred:[0 0 0 0 0 0 0 1]
True:[0 1 0 0 0 1 0 1]
9 + 60 = 1
------------
Error:[3.63389116]
Pred:[1 1 1 1 1 1 1 1]
True:[0 0 1 1 1 1 1 1]
28 + 35 = 255
------------
Error:[3.91366595]
Pred:[0 1 0 0 1 0 0 0]
True:[1 0 1 0 0 0 0 0]
116 + 44 = 72
------------
Error:[3.72191702]
Pred:[1 1 0 1 1 1 1 1]
True:[0 1 0 0 1 1 0 1]
4 + 73 = 223
------------
Error:[3.5852713]
Pred:[0 0 0 0 1 0 0 0]
True:[0 1 0 1 0 0 1 0]
71 + 11 = 8
------------
Error:[2.53352328]
Pred:[1 0 1 0 0 0 1 0]
True:[1 1 0 0 0 0 1 0]
81 + 113 = 162
------------
Error:[0.57691441]
Pred:[0 1 0 1 0 0 0 1]
True:[0 1 0 1 0 0 0 1]
81 + 0 = 81
------------
Error:[1.42589952]
Pred:[1 0 0 0 0 0 0 1]
True:[1 0 0 0 0 0 0 1]
4 + 125 = 129
------------
Error:[0.47477457]
Pred:[0 0 1 1 1 0 0 0]
True:[0 0 1 1 1 0 0 0]
39 + 17 = 56
------------
Error:[0.21595037]
Pred:[0 0 0 0 1 1 1 0]
True:[0 0 0 0 1 1 1 0]
11 + 3 = 14
------------


# IMDB reviews (keras)

In [87]:
from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [91]:
max_features = 50000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 150
batch_size = 50 # увеличьте значение для ускорения обучения

In [92]:
print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

Загрузка данных...


In [93]:
len(x_train[0])

218

In [94]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 150)
x_test shape: (25000, 150)


In [95]:
layer = Embedding(max_features, 128)

In [96]:
layer(x_train[0]).shape

TensorShape([150, 128])

In [97]:
print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Построение модели...


In [98]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [99]:
print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1, # увеличьте при необходимости
          validation_data=(x_test, y_test))

Процесс обучения...
500/500 [==============================] - 751s 1s/step - loss: 0.3916 - accuracy: 0.8255 - val_loss: 0.3887 - val_accuracy: 0.8202


In [100]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

500/500 [==============================] - 94s 188ms/step - loss: 0.3887 - accuracy: 0.8202


In [101]:
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

Результат при тестировании: 0.38871851563453674
Тестовая точность: 0.8201599717140198


1) Без изменений \
Результат при тестировании: 0.3669951260089874 \
Тестовая точность: 0.8393999934196472 

2) max_features с 20т до 50т слов \
Результат при тестировании: 0.3569040298461914 \
Тестовая точность: 0.8406400084495544 \
224s 442ms/step

3) maxlen(длина отзыва до обрезания) с 80 до 150 слов \
Результат при тестировании: 0.33390212059020996 \
Тестовая точность: 0.858240008354187 \
499s 992ms/step

4) 3 эпохи \
Результат при тестировании: 0.43001750111579895 \
Тестовая точность: 0.8362799882888794 \
~800cек

5) dropout до 0.1 \
Результат при тестировании: 0.4021052420139313 \
Тестовая точность: 0.8237199783325195 \
237s 469ms/step

Увеличение словаря с 20 до 50 тысяч слов не сильно отразилось на метриках. Видимо, настолько редкие слова не особо важны. Увеличение длины отзыва почти в два раза сильно замедлило обучение, но прирост качества оказался незначительным. Увеличение количества эпох привело к явному переобучению модели. Потом вернул одну эпоху и уменьшил dropout до 0.1. Вернулся примерно к начальным значениям метрики. Вернул все как было, поставил relu вместо сигмоиды. Результат упал аж до 74%. В итоге чуть-чуть поднять метрику удалось только за счет увеличения длины отзыва до обрезания. Время на обучение увеличивается значительно, но есть хоть сколь либо заметный прирост качества, с ~84% до ~86%. (результат 3 в сводке выше)

# Генерация текста на основе книжки «Алиса в стране чудес»

In [1]:
import numpy as np
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.models import Sequential

In [2]:
# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set(text)
nb_chars = len(chars)

In [4]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [24]:
#index2char

In [68]:
# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 20, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [69]:
len(input_chars)

143494

In [70]:
nb_chars

45

In [71]:
input_chars[0], label_chars[0]

("alice's adventures i", 'n')

In [72]:
input_chars[1], label_chars[1]

("lice's adventures in", ' ')

In [73]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y
# Х - трехмерный one-hot массив с осями: все последовательности, длина одной последовательности,45-значный one-hot код для каждой буквы.
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [74]:
X.shape

(143494, 20, 45)

In [75]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 256
NUM_ITERATIONS = 5 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [76]:
'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. 
За ним последует полносвязный слой связанный с набором возможных следующих символов, 
которые конвертированы в вероятностные результаты через стандартную softmax активацию 
с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=True,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)

model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=True,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)

model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        unroll=True
    )
)

model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [77]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred

Итерация #: 0
1122/1122 [==============================] - 724s 629ms/step - loss: 2.0027
Генерация из посева: live the strange cre
live the strange creat of the rambout of the rambout of the rambout of the rambout of the rambout of the rambout of the ==================================================
Итерация #: 1
1122/1122 [==============================] - 687s 612ms/step - loss: 1.5060
Генерация из посева: ' `oh, a song, pleas
' `oh, a song, pleased to the mouse of the set was and the mouse of the set was and the mouse of the set was and the mou==================================================
Итерация #: 2
1122/1122 [==============================] - 671s 598ms/step - loss: 1.3231
Генерация из посева: as that you had been
as that you had been the tried to the tried to the tried to the tried to the tried to the tried to the tried to the trie==================================================
Итерация #: 3
1122/1122 [==============================] - 693s 618ms/step - loss: 1.2124
Г

добавил еще один GRU слой, увеличил размер внутреннего состояния до 256, удилинил строку до 20 символов. Уменьшил количество итераций до 5, потому что одна итерация - 11 минут)

самая осмысленная фраза - "fan and two shook it in a conversation. `that's all the reason of much share shouted the queen said to herself" Но это видимо случайно получилось. 